# TP Coding Convolutional Neural Networks in tensorflow and keras - part 2

Author : Alasdair Newson

alasdair.newson@telecom-paris.fr

In this session, we shall be looking at two subjects :

- A way to visualise what networks are learning : the Deep Dream algorithm
- Adversarial examples

At the heart of these two applications is the calculation of the gradient of a loss function with respect to the image itself (instead of respect to the weights). The loss function will be defined depending on the application at hand.

We shall use a pre-trained, well-known pretrained network. The lab has been designed for either VGG16 or inceptionv3. You can try with other networks, but bear in mind that the pre-processing operations are different for each network, so you would have to change the code correspondingly.

We can easily access certain well-known networks with the Tensorflow programming framework. There are useful predefined functions which allow us to load the weights, view the architecture etc. of the networks. We will specify these functions as necessary through the lab work. Unfortunately, the documentation for these functions is not very plentiful, but if you want to look at exactly what they do, you can look at the source code for help :

https://github.com/keras-team/keras-applications/blob/master/keras_applications/vgg16.py

https://github.com/keras-team/keras-applications/blob/master/keras_applications/inception_v3.py

https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py

However, we will indicate how these functions work as the need arises.


First, let's load the necessary packages.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.layers import Input
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

Now, let's load one of the most famous networks, VGG16 or inceptionv3 (you choose), and view it's architecture with the summary() function.

In [ ]:

model_name = 'inceptionv3'#'vgg16'#

if model_name == 'inceptionv3':
	model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
elif model_name == 'vgg16':
	model = tf.keras.applications.VGG16(include_top=False, weights='imagenet')

model.summary()


You can also extract the weights of the convolutional filters in the deep CNN. You can do this with the following function :

- weights = layer.get_weights()

where 'layer' iterates throughout the 'model.layers' list. Note that this function returns the following :
- [weights, biases] if the layer is convolutional. weights has size [m,n,filter_depth,n_filters]
- an empty list otherwise

Write a function which retrieves the weights of the network (you will have to test if the weights variable is empty at each iteration).
    

In [ ]:
def retrieve_weights(model):
    weight_list = []
    # BEGIN CODE HERE
    # END CODE HERE
    return weight_list

Now, write a code that visualises a single channel of a filter of your choice. Reminder, to view a grey-level image, you can use :

- plt.imshow(img,cmap="gray")
- plt.show()

In [ ]:
weight_list = retrieve_weights(model)
curr_weight = weight_list[2]
curr_weight.shape
plt.imshow(curr_weight[:,:,0,1],cmap="gray")

In [ ]:
curr_weight[:,:,0,1]

As you can probably see, this visualisation is not of much use : we cannot really tell what is going on in the network. For this, let's turn to another approach : Deep Dream !

## 2. Deep Dream

We now proceed to carry out the Deep Dream algorithm. The idea of the Deep Dream algorithm is to find an image which maximises the response of a network at a certain layer : $\textbf{this should help us understand what the network is learning}$. This can be done with an iterative algorithm, by simply carrying out gradient $\textbf{ascension}$. We start with an input image and iteratively add the gradient of the average response of the features which interest us. A pseudo-code for this would be :

- img = img_in
- for i=1:n_iters
    - img = img + grad_step $\nabla_{img} \mathcal{L}$,
    
where $\mathcal{L}$ is a function of the responses which interest us (you need to define this). In this part of the lab, we shall use the average response.

Let's first define a function to preprocess the image. This is needed to put the image in the correct format for the VGG16 or inceptionv3 networks. We also create a function to invert this process. You can add cases for other networks if you wish (and you can get it working :) ).

In [ ]:
def format_image(img_file,model_name):
	"""
	This function reads and formats an image so that it can be fed to the VGG16 or inceptionv3 networks
	
	Parameters
	----------
	img_file : image file name
	
	Returns
	-------
	img_out_model : the correctly formatted image for VGG16 or inceptionv3 networks
	img : the image as read by the load_img function of keras.preprocessing.image
	"""
	# read image
	img = load_img(img_file)
	# convert image to an array
	img_out = img_to_array(img)
	# preprocess the image to put in the correct format for use with the VGG16/inceptionv3 network trained on imagenet
	
	if model_name == 'inceptionv3':
		#https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input
		#The inputs pixel values are scaled between -1 and 1, sample-wise.
		img_out_model = tf.keras.applications.inception_v3.preprocess_input(img_out)
	elif model_name == 'vgg16':
		#https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input
		#The images are converted from RGB to BGR,
		#then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.
		img_out_model = tf.keras.applications.vgg16.preprocess_input(img_out)
	# add a dimension at the beginning, coresponding to the batch dimension
	img_out_model = np.expand_dims(img_out_model, axis=0)
	return img_out_model, img

def unformat_image(img_in,model_name):
	"""
	This function inverts the preprocessing applied to images for use in the VGG16/inceptionv3 network
	
	Parameters
	----------
	img_file : formatted image of shape (batch_size,m,n,3)
	
	Returns
	-------
	img_out : a m-by-n-by-3 array, representing an image that can be written to an image file
	"""
	#get rid of batch dimension
	img_out=np.squeeze(img_in)

	
	if(model_name == 'inceptionv3'):
		#clamp image to the range [0,255] and cast to uint8
		img_out = 255*(img_out + 1.0)/2.0
	if (model_name == 'vgg16'):
		#https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input
		#The images are converted from RGB to BGR,
		#then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.
		#remove offsets added by the VGG16 preprocessing
		img_out[:, :, 0] += 103.939
		img_out[:, :, 1] += 116.779
		img_out[:, :, 2] += 123.68
		#invert the order of the colours : BGR -> RGB
		img_out = img_out[:, :, ::-1]
		#img_out = 255*(img_out + 1.0)/2.0
	
	img_out = np.clip(img_out, 0, 255).astype('uint8')
	return img_out

Now, we load the image. At the same time, we create a backend Tensor which has the correct format for the network.

**NOTE** : if you are using colab, then you might not be able to easily upload the images of the lab work. In this case you can use download the image directly from the url given (this is done for you). To modify the behaviour, change the ```using_colab``` variable.

In [ ]:
using_colab = True

#load image
if (using_colab == True):
    !wget "https://perso.telecom-paristech.fr/anewson/doc/images/got.jpg"
    img_in,img_visu = format_image('got.jpg',model_name)
else:
    img_in,img_visu = format_image('images/got.jpg',model_name)
#show the input image
plt.imshow(img_visu)
plt.show()

Now, we define the loss that we wish to maximise. This can be anything you wish, but a common loss is simply the average response of a certain channel of a certain layer. Since these responses are all positive, due to the non-linearities used, we can safely take the average as a loss function to maximise.

We are going to use the responses of different layers depending on whether we use inceptionv3 or vgg16. You can modify the names of the layers to try and inspect different layers.


In [ ]:

if(model_name == 'inceptionv3'): 
	layer_names = ['mixed3', 'mixed5']
elif (model_name == 'vgg16'):
	layer_names = ['block5_conv2', 'block5_conv3']
layers = [model.get_layer(name).output for name in layer_names]
# create a model which outputs the layer which interests us
img_backend = model.input
deepdream_model = tf.keras.Model(inputs=img_backend, outputs=layers)
print(deepdream_model)


Now, we create the deep dream loss. 

The operations which calculate the loss should be done using the tensorflow functionalities, since we are working on symbolic Tensors. For example, use the following function for calculateing the mean :
- tf.math.reduce_mean()
and for calculating the total loss 
- tf.math.reduce_sum()

In [ ]:
def deep_dream_loss(model_in,img_in):
	activation = model_in(img_in) 
	if len(activation) == 1:
		activation = [activation]

	losses = []
	for act in activation:
		loss = ... # FILL IN CODE
		losses.append(loss)
	total_loss = ... # FILL IN CODE

	return total_loss

Now, we create a function to carry out one step of the gradient __ascent__. For this, we need to calculate the gradient of the loss with respect to the input image. 

This is done with the following syntax. For this example, we consider that ```x``` is the variable, and ```my_loss_function``` is the function which we want to take the derivative of.

```
with tf.GradientTape() as tape:
    tape.watch(x)
    loss = my_loss_function(x)
grads = tape.gradient(loss,x)
```

Next, we use a little trick. Indeed, it may be the case that the gradients are far too small or far too large for updating, meaning that the updates do nothing or destroy the image. To avoid this, we normalise the gradients, such that the standard deviation is 1. For this, use the following function :

- tf.math.reduce_std()

Finally, to make sure there are no division problems, we also add a small constant $\epsilon $to the standard deviation. In other words :

- $\nabla = \frac{\nabla}{std(\nabla) + \epsilon}$

Set this constant to 1e-8.

Fill in the following function which carries out a step of the gradient ascent.

In [ ]:
def gradient_step(model_in, img_in, step_size, model_name):
  # BEGIN STUDENT CODE

  # END STUDENT CODE
  img_in = ... # FILL IN CODE : CARRY OUT A STEP OF THE GRADIENT ASCENT

  if (model_name == 'inceptionv3'):
    img_in = tf.clip_by_value(img_in, -1, 1) # special case for inceptionv3
  return img_in, loss
  #END STUDENT CODE

We are now ready to carry out the Deep Dream algorithm using gradient ascent, yipee ! Iterate 'n_iterations' times, each time adding an epsilon of the gradient. 

In [ ]:
# first, reload image to make sure that we are not starting from a previous initialisation
if (using_colab == True):
	#!wget 'https://perso.telecom-paristech.fr/anewson/doc/images/got.jpg'
	img_in,_ = format_image('got.jpg',model_name)
else:
	img_in,_ = format_image('images/got.jpg',model_name)

img_in = tf.convert_to_tensor(img_in)
if (model_name == 'inceptionv3'):
	step = 0.1 # Gradient ascent step size
elif (model_name == 'vgg16'):
	step = 0.5 # Gradient ascent step size
n_iterations = 40  # Number of gradient ascent steps

for ii in range(0,n_iterations):
	img_in, loss = ... # FILL IN STUDENT
	
	if (ii%5==0):
		print(".", end='')
		img_out = unformat_image(np.copy(img_in),model_name)
		plt.figure(num=None, figsize=(10, 8))
		plt.imshow(img_out)
		plt.show()
    #plt.imsave('img_out_'+str(ii).zfill(3)+'.png',img_out)

You can try different layers and switch between the two models and see what the results are !

## 3. Adversarial examples

In this part of the lab work, we will explore the interesting case of adversarial examples. Adversarial examples are images which have been perturbed in a manner which __makes the network misclassify the image__.

There are many ways to do this, however we can use a similar approach to the one used above, that is to say, we will use a gradient maximisation approach. This consists in iteratively adding the gradient of the loss with respect to the image, to the current image, in order to get a misclassified image.

For this application, we need access to the last, classification, layer of the network. To do this, we set the ```include_top``` variable to ```True```.

In [ ]:
model_name = 'vgg16'#'inceptionv3'#
# load model, including last (classification layer)
if model_name == 'inceptionv3':
	model = tf.keras.applications.InceptionV3(include_top=True, weights='imagenet')
elif model_name == 'vgg16':
	model = tf.keras.applications.VGG16(include_top=True, weights='imagenet')

# display architecture
model.summary()

You may have noticed an increase of the number of parameters in the network. This is normal, since we have included the last layer, the classification layer. This is a dense layer(s) and therefore has many parameters.


We are nw going to take an image of a cat and try to misclassify it. First, read and format the image.

NOTE !!!

In the previous code, we were not interested in the dense layers, so the input image could be of __any size__ (there were only convolutional and maxpool layers). Now, since we are using the fully connected layers, we need to make sure the input is of the correct size for the images used in each network. For this, we redefine the format_image function to force the image to a certain size, depending on the network.

In [ ]:
def format_image_classif(img_file,model_name):
    """
    This function reads and formats an image so that it can be fed to the VGG16 or InceptionV3 network.
    In this case, we wish to force the image size to a certain shape, since we want to use the image for
    classification
    
    Parameters
    ----------
    img_file : image file name
    img_width : the target image width
    img_height : he target image height
    
    Returns
    -------
    img_out_model : the correctly formatted image for VGG16 or InceptionV3
    img : the image as read by the load_img function of keras.preprocessing.image
    """
    if model_name == 'inceptionv3':
      img_width = 299
      img_height = 299
    elif model_name == 'vgg16':
      img_width = 224
      img_height = 224
    # read image. Force the image size to a certain shape (uses a resize of the pillow package)
    img = load_img(img_file,target_size=(img_height,img_width))
    # convert image to an array
    img_out = img_to_array(img)

    if model_name == 'inceptionv3':
      #https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input
      #The inputs pixel values are scaled between -1 and 1, sample-wise.
      img_out_model = tf.keras.applications.inception_v3.preprocess_input(img_out)
    elif model_name == 'vgg16':
      #https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input
      #The images are converted from RGB to BGR,
      #then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.
      img_out_model = tf.keras.applications.vgg16.preprocess_input(img_out)
    # add a dimension at the beginning, coresponding to the batch dimension
    img_out_model = np.expand_dims(img_out_model, axis=0)
    return img_out_model, img

In [ ]:
if (using_colab == True):
    !wget "https://perso.telecom-paristech.fr/anewson/doc/images/cat_small.png"
    img_in,img_visu = format_image_classif('cat_small.png',model_name)
else:
    img_in,img_visu = format_image_classif('images/cat_small.png',model_name)

plt.imshow(img_visu)
print(img_in.shape)

Now, we need to retrieve the last layer of the VGG or inceptionv3 networks. Do this in the same manner as above, but changing the name of the layer. To find the right name, look at the architecutre displayed above.

In [ ]:
if(model_name == 'inceptionv3'): 
	layer_names = ...	# FILL IN STUDENT
elif (model_name == 'vgg16'):
	layer_names = ...	# FILL IN STUDENT
layers = [model.get_layer(name).output for name in layer_names]
# create a model which outputs the layer which interests us
img_backend = model.input
adversarial_model = tf.keras.Model(inputs=img_backend, outputs=layers)


We are going to try to force the image to recognise a 'reflex_camera'. This is number 759 of the imagenet classes. You can use any one you like in fact (apart from ones linked to cats, obviously). To see the list of classes go to :

https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

Now, find the initial class of the cat image. Also display the top 5 classification results. For this, you can use the following functions :

- model.predict(img) : classification prediction of the img variable
- ```tf.keras.applications.vgg16.decode_predictions()``` or ```tf.keras.applications.inception_v3.decode_predictions()``` : converts the numerical probabilities in the y variable to human readable classes.

In [ ]:
target_class = 759
# carry out the network predictions on the example image
y_predicted = ... # FILL IN STUDENT
#define the true class as the initial most likely class
true_class = np.zeros((1,y_predicted.shape[1]))
true_class_ind = ... # FILL IN STUDENT
true_class[0,true_class_ind] = 1 
# print the "true" class, as well as the top 5 predicted classes, with the prediction probability
if model_name == 'inceptionv3':
  print("True class : ",...)
  y_predicted_decoded =... # FILL IN STUDENT
elif model_name == 'vgg16':
  print("True class : ",...)
  y_predicted_decoded = ... # FILL IN STUDENT
print(y_predicted_decoded)
# END STUDENT CODE

We must redefine the loss function for the adversarial model. Careful, the first axis of the activation is the batch dimension, the second one corresponds to the different categories.

You must also redefine the gradient step function, so that it calls the new adversarial loss, and so that it takes a new parameter ```target_class``` as an input (other than this, the gradient step function is not modified, so you can look at your previous code as an example).

In [ ]:
def adversarial_loss(model_in,img_in,target_class):
	activation = model_in(img_in) 

	total_loss = ...    # FILL IN STUDENT

	return total_loss

def adversarial_gradient_step(model_in, img_in, step_size, model_name,target_class):
  # BEGIN STUDENT CODE
  
  img_in = ...  # FILL IN STUDENT

  # END STUDENT CODE
  if (model_name == 'inceptionv3'):
    img_in = tf.clip_by_value(img_in, -1, 1) # special case for inceptionv3
  return img_in, loss

We are now ready to perturb the image such that we misclassify it. Youhoo !

In [ ]:
#reload image, to make sure we are not starting from previous point
if (using_colab == True):
    !wget "https://perso.telecom-paristech.fr/anewson/doc/images/cat_small.png"
    img_in,img_visu = format_image_classif('cat_small.png',model_name)
else:
    img_in,img_visu = format_image_classif('images/cat_small.png',model_name)

#img_in = 10*np.random.randn(1,img_in.shape[1],img_in.shape[2],img_in.shape[3])
#parameters
img_in = tf.convert_to_tensor(img_in)
if (model_name == 'inceptionv3'):
  step = 0.01  # Gradient ascent step size
elif(model_name == 'vgg16'):
  step = 0.1  # Gradient ascent step size
n_iterations = 100  # Number of gradient ascent steps
for ii in range(0,n_iterations):
    img_in, loss = ... # FILL IN STUDENT
    if (ii%5==0):
        img_show = unformat_image(np.copy(img_in),model_name)
        plt.imsave('img_out_'+str(ii).zfill(3)+'.png',img_show)
        #predict current model to see evolution of top classification
        y_predicted = model.predict(img_in)
        if (model_name == 'inceptionv3'):
          print(tf.keras.applications.inception_v3.decode_predictions(y_predicted, top=1)[0])
        if (model_name == 'vgg16'):
          print(tf.keras.applications.vgg16.decode_predictions(y_predicted, top=1)[0])
print('End of optimisation')
# finally, display misclassified image
img_show = unformat_image(np.copy(img_in),model_name)
plt.figure(num=None, figsize=(10, 8))
plt.imshow(img_show)
#show the final top 5 classes
y_predicted = model.predict(img_in)
if (model_name == 'inceptionv3'):
  print(tf.keras.applications.inception_v3.decode_predictions(y_predicted, top=5)[0])
if (model_name == 'vgg16'):
  print(tf.keras.applications.vgg16.decode_predictions(y_predicted, top=5)[0])

As you should probably see, the image is changed such that it is no longer is correctly classified. It should be classified as a 'reflex camera', or whatever you chose, with high probability. This is a problem, since a human is still able to see a cat ! Furthermore, the top 5 classifications have nothing to do with cats !! Even bigger problem !!!

# Evaluation

To evaluate the work, you should rate the code for :
- 1) Deep dream, ```retrieve_weights``` function : 2 points
- 2) Deep dream, ```deep_dream_loss``` function : 1 point
- 3) Deep dream, ```gradient_step``` function : 2 points
- 4) Adversarial examples, choosing the right layer for the loss : 1 point
- 5) Adversarial examples, ```y_predicted```, ```true_class``` and ```y_predicted_decoded``` : 3 Points
- 6) Adversarial examples,  ```adversarial_loss``` function : 1 point

Total over 10 points.